# Dubai — Quantum accurate bond inference and partial charge calculations

See [the tutorial](/Tutorials/ligand_bond_inference_and_partial_charge_calculation.ipynb).

In [ ]:
# |hide
import os
import pathlib

WORK_DIR = pathlib.Path("~/qdx/dubai_quickstart/").expanduser()
if WORK_DIR.exists():
    !rm -r $WORK_DIR
os.makedirs(WORK_DIR)
os.chdir(WORK_DIR)

In [ ]:
import json
from pathlib import Path

import requests
import rush

# Get our client, for calling modules and using the rush API
# set the RUSH_TOKEN  environment variables
client = rush.build_blocking_provider_with_functions(
    batch_tags=["bond_inference_quickstart"]
)

# Convert aspirin to a QDXF file so we can use it for this demo
SMILES_STRING = "CC(=O)OC1=CC=CC=C1C(=O)O"
SDF_LINK = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/{SMILES_STRING}/record/SDF?record_type=3d"

file_path = client.workspace / "aspirin.sdf"
with open(file_path, "wb") as f:
    f.write(requests.get(SDF_LINK).content)

(ligand,) = client.convert("SDF", Path(file_path))

ligand = json.load(open(ligand.download(), "r"))

# Remove connectivity
EXPECTED_CONNECTIVITY = ligand[0]["topology"]["connectivity"]
ligand = ligand[0]
del ligand["topology"]["connectivity"]

ligand["topology"]["fragment_multiplicities"] = [1]

# Arguments
DUBAI_RESOURCES = {
    "gpus": 1,
    "storage": 1024_000,
    "walltime": 60,
}
LIGAND_FILEPATH = client.workspace / "aspirin.qdxf.json"
json.dump(ligand, open(LIGAND_FILEPATH, "w"))

# Run the inference
(ligand_with_bonds,) = client.dubai(LIGAND_FILEPATH, resources=DUBAI_RESOURCES)

output_ligand = json.load(ligand_with_bonds.download().open())

for expected_bond, outputted_bond in zip(
    EXPECTED_CONNECTIVITY, output_ligand["topology"]["connectivity"]
):
    # Check start atoms are the same
    assert expected_bond[0] == outputted_bond[0]
    # Check ending atoms are the same
    assert expected_bond[1] == outputted_bond[1]
    # NB: we don't check the third item of the bond -- the bond type. This is
    # because Dubai accurately outputs ring bonds as a specific 'RINGBOND' type,
    # whereas SDF aspirin was interleaving single and double bonds.
print("Bond inference performed correctly!")

2024-03-20 16:44:32,219 - rush - INFO - Restoring by default via env
2024-03-20 16:44:35,065 - rush - INFO - Trying to restore job with tags: ['bond_inference_quickstart'] and path: github:talo/tengu-prelude/b345a0b0077225c63d904d0e03fb9ca1acec55ed#convert
2024-03-20 16:44:35,124 - rush - INFO - Restoring job from previous run with id 1dbfa498-4739-49a2-80a2-c53d98691f39
2024-03-20 16:44:37,372 - rush - INFO - Argument 6fb8de19-33c5-4e11-b777-6aeab1f51512 is now ModuleInstanceStatus.RESOLVING
2024-03-20 16:44:42,945 - rush - INFO - Argument 6fb8de19-33c5-4e11-b777-6aeab1f51512 is now ModuleInstanceStatus.ADMITTED
2024-03-20 16:44:55,162 - rush - INFO - Argument 6fb8de19-33c5-4e11-b777-6aeab1f51512 is now ModuleInstanceStatus.DISPATCHED
2024-03-20 16:45:00,789 - rush - INFO - Argument 6fb8de19-33c5-4e11-b777-6aeab1f51512 is now ModuleInstanceStatus.RUNNING
2024-03-20 16:45:11,115 - rush - INFO - Argument 6fb8de19-33c5-4e11-b777-6aeab1f51512 is now ModuleInstanceStatus.AWAITING_UPLOAD
Bo